In [1]:
import sqlite3
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
from matplotlib.colors import TABLEAU_COLORS
import seaborn as sns

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from haversine import haversine, Unit
import pulp
from geopy.distance import geodesic
import networkx as nx
from networkx.algorithms.approximation import greedy_tsp
from folium.plugins import AntPath

In [2]:
conn = sqlite3.connect("sales_agent_co_pilot.db")

In [3]:
retailers_df = pd.read_sql_query("""SELECT * FROM retailers;""", conn)
retailer_map_df = pd.read_sql_query("SELECT * FROM retailer_beat_map", conn)

In [4]:
# Center of Retailer Universe
map_center = [
    retailers_df["Latitude"].mean(),
    retailers_df["Longitude"].mean()
]

In [5]:
# Create map
m = folium.Map(location=map_center, zoom_start=12)
marker_cluster = MarkerCluster().add_to(m)

for _, row in retailers_df.iterrows():
    popup = f"{row['Name']} ({row['Channel']})"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        tooltip=row["Retailer_ID"]
    ).add_to(marker_cluster)

In [6]:
m

In [7]:
# Merge to get lat/lon per beat
retailers_merged = pd.merge(retailers_df, retailer_map_df, on="Retailer_ID")

# Generate color map for beats (12 unique colors)
beat_ids = sorted(retailers_merged["Beat_ID"].unique())
#colors = list(TABLEAU_COLORS.values())
colors = sns.color_palette("hls", 12).as_hex()
beat_color_map = {beat_id: colors[i % len(colors)] for i, beat_id in enumerate(beat_ids)}

In [8]:
m = folium.Map(location=map_center, zoom_start=12)

# Add retailers with color by beat
for _, row in retailers_merged.iterrows():
    beat = row["Beat_ID"]
    color = beat_color_map[beat]
    popup = f"Retailer: {row['Name']}<br>Beat: {beat}"
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=popup,
        tooltip=row[['Retailer_ID','Beat_ID']]
    ).add_to(m)

m

#### beat optimization

In [9]:
# Parameters
NUM_BEATS = 12
MIN_BEAT_SIZE = 16
MAX_BEAT_SIZE = 24

In [10]:
retailer_ids = retailers_df["Retailer_ID"].tolist()
coords = retailers_df[["Latitude", "Longitude"]].to_numpy()
num_retailers = len(retailer_ids)

In [11]:
#Use KMeans to get initial cluster centers
kmeans = KMeans(n_clusters=NUM_BEATS, random_state=42).fit(coords)
initial_centers = kmeans.cluster_centers_

In [12]:
#Calculate distance matrix (Haversine in kilometers)
dist_matrix = np.zeros((num_retailers, NUM_BEATS))

for i in range(num_retailers):
    for j in range(NUM_BEATS):
        latlon_i = tuple(coords[i])  # (lat, lon)
        latlon_center = tuple(initial_centers[j]) 
        dist_matrix[i][j] = haversine(latlon_i, latlon_center, unit=Unit.KILOMETERS)

In [13]:
#Set up pulp optimization
model = pulp.LpProblem("ConstrainedClustering", pulp.LpMinimize)

# Binary decision variables: x[i][j] = 1 if retailer i assigned to cluster j
x = [
    [pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(NUM_BEATS)]
    for i in range(num_retailers)
]

In [14]:
# Objective: Minimize total Haversine distance
model += pulp.lpSum(dist_matrix[i][j] * x[i][j] for i in range(num_retailers) for j in range(NUM_BEATS))

In [15]:
# Constraint 1: Each retailer must be assigned to exactly one beat
for i in range(num_retailers):
    model += pulp.lpSum(x[i][j] for j in range(NUM_BEATS)) == 1

In [16]:
# Constraint 2: Cluster sizes must be within bounds
for j in range(NUM_BEATS):
    model += pulp.lpSum(x[i][j] for i in range(num_retailers)) >= MIN_BEAT_SIZE
    model += pulp.lpSum(x[i][j] for i in range(num_retailers)) <= MAX_BEAT_SIZE

In [17]:
#solve the optimization problem
model.solve(pulp.PULP_CBC_CMD(msg=1))

1

In [18]:
#Extract assignments
assignments = []
for i in range(num_retailers):
    for j in range(NUM_BEATS):
        if pulp.value(x[i][j]) == 1:
            assignments.append((retailer_ids[i], f"B{j+1:03d}"))
            break

In [19]:
optimized_beatmap_df = pd.DataFrame(assignments, columns=["Retailer_ID", "Beat_ID"])

In [20]:
optimized_beatmap_df['Retailer_ID'].nunique(),optimized_beatmap_df['Beat_ID'].nunique()

(240, 12)

In [21]:
optimized_beatmap_df.head(),retailer_map_df.head()

(  Retailer_ID Beat_ID
 0       R0001    B012
 1       R0002    B004
 2       R0003    B008
 3       R0004    B007
 4       R0005    B009,
   Retailer_ID Beat_ID
 0       R0001    B004
 1       R0002    B007
 2       R0003    B010
 3       R0004    B006
 4       R0005    B010)

In [22]:
optimized_beatmap_df.to_sql("retailer_beat_map_optimized", conn, if_exists="replace", index=False)

240

In [23]:
retailers_merged_optimized = pd.merge(retailers_df, optimized_beatmap_df, on="Retailer_ID")

# Generate color map for beats (12 unique colors)
beat_ids = sorted(retailers_merged_optimized["Beat_ID"].unique())
#colors = list(TABLEAU_COLORS.values())
colors = sns.color_palette("hls", NUM_BEATS).as_hex()
beat_color_map = {beat_id: colors[i % len(colors)] for i, beat_id in enumerate(beat_ids)}

In [24]:
len(colors)

12

In [25]:
m = folium.Map(location=map_center, zoom_start=12)

# Add retailers with color by beat
for _, row in retailers_merged_optimized.iterrows():
    beat = row["Beat_ID"]
    color = beat_color_map[beat]
    popup = f"Retailer: {row['Name']}<br>Beat: {beat}"
    
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=popup,
        tooltip=row[['Retailer_ID','Beat_ID']]
    ).add_to(m)

m

In [26]:
optimized_beatmap_df['Beat_ID'].value_counts()

Beat_ID
B012    24
B004    24
B007    23
B005    23
B002    23
B001    22
B009    19
B003    17
B006    17
B008    16
B011    16
B010    16
Name: count, dtype: int64

In [27]:
def compute_tsp_route(beat_id, beat_retailers):
    coords = beat_retailers[["Latitude", "Longitude"]].to_numpy()
    retailer_ids = beat_retailers["Retailer_ID"].tolist()
    n = len(retailer_ids)

    # Build full distance matrix (in km)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = geodesic(coords[i], coords[j]).km
            dist_matrix[i][j] = dist_matrix[j][i] = d

    # Find outermost retailer (farthest from centroid)
    centroid = coords.mean(axis=0)
    dists_to_center = [geodesic(tuple(c), tuple(centroid)).km for c in coords]
    start_idx = int(np.argmax(dists_to_center))

    # Build complete graph with distance weights
    G = nx.complete_graph(n)
    for i in range(n):
        for j in range(n):
            if i != j:
                G[i][j]["weight"] = dist_matrix[i][j]

    # Approximate TSP without 'start' (avoid TypeError)
    #tsp_path = nx.approximation.traveling_salesman_problem(G, cycle=False, weight="weight", method="greedy")

    tsp_path = greedy_tsp(G, weight="weight")

    # Return final path as list of tuples
    return [(beat_id, retailer_ids[i], seq + 1) for seq, i in enumerate(tsp_path)]
    return [(beat_id, retailer_ids[i], seq + 1) for seq, i in enumerate(tsp_path)]


In [28]:
# Run TSP for each beat
all_routes = []
for beat_id in retailers_merged_optimized["Beat_ID"].unique():
    beat_retailers = retailers_merged_optimized[retailers_merged_optimized["Beat_ID"] == beat_id].reset_index(drop=True)
    route = compute_tsp_route(beat_id, beat_retailers)
    all_routes.extend(route)

In [29]:
route_df = pd.DataFrame(all_routes, columns=["Beat_ID", "Retailer_ID", "Visit_Sequence"])
route_df.head()

,Beat_ID,Retailer_ID,Visit_Sequence
0,B012,R0001,1
1,B012,R0141,2
2,B012,R0053,3
3,B012,R0218,4
4,B012,R0199,5


In [30]:
route_df['Beat_ID'].nunique()

12

In [31]:
route_df = pd.DataFrame(all_routes, columns=["Beat_ID", "Retailer_ID", "Visit_Sequence"])
route_df.to_sql("beat_route_plan", conn, if_exists="replace", index=False)
conn.commit()
# conn.close()

In [32]:
conn.close()

In [33]:
route_merged = pd.merge(route_df, retailers_df, on="Retailer_ID")
route_merged = route_merged.sort_values(by=["Beat_ID", "Visit_Sequence"])

# Create map centered on Bengaluru
center_lat, center_lon = retailers_df["Latitude"].mean(), retailers_df["Longitude"].mean()

In [34]:
beat_ids = sorted(route_merged["Beat_ID"].unique())
beat_color_map = {beat_id: colors[i % len(colors)] for i, beat_id in enumerate(beat_ids)}

In [35]:
print(route_merged[["Beat_ID", "Retailer_ID", "Visit_Sequence"]].sort_values(["Beat_ID", "Visit_Sequence"]).head(20))
print(route_merged["Visit_Sequence"].dtype)

    Beat_ID Retailer_ID  Visit_Sequence
128    B001       R0010               1
129    B001       R0171               2
130    B001       R0212               3
131    B001       R0210               4
132    B001       R0161               5
133    B001       R0043               6
134    B001       R0238               7
135    B001       R0104               8
136    B001       R0036               9
137    B001       R0069              10
138    B001       R0229              11
139    B001       R0101              12
140    B001       R0231              13
141    B001       R0139              14
142    B001       R0040              15
143    B001       R0143              16
144    B001       R0065              17
145    B001       R0046              18
146    B001       R0207              19
147    B001       R0090              20
int64


In [36]:
m = folium.Map(location=map_center, zoom_start=12)

# Plot ant path for each beat
for beat_id in beat_ids:
    beat_route = route_merged[route_merged["Beat_ID"] == beat_id]
    locations = beat_route[["Latitude", "Longitude"]].values.tolist()

    AntPath(
        locations,
        color=beat_color_map[beat_id],
        weight=4,
        opacity=0.8,
        dash_array=[10, 20]
    ).add_to(m)

    
    for i, row in beat_route.iterrows():
        folium.CircleMarker(
            location=(row["Latitude"], row["Longitude"]),
            radius=3,
            fill=True,
            fill_opacity=0.9,
            popup=f"{row['Retailer_ID']} - {row['Visit_Sequence']}",
            color=beat_color_map[beat_id]
        ).add_to(m)

        folium.map.Marker(
        [row["Latitude"], row["Longitude"]],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 10pt; color: black"><b>{row['Visit_Sequence']-1}</b></div>"""
        )
        ).add_to(m)

m